In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from types import SimpleNamespace
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.io import wavfile
from scipy.signal import correlate
import matplotlib.pyplot as plt


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/test"))

# Any results you write to the current directory are saved as output.

['F4.wav', 'M3.wav', 'F2.wav', 'F3.wav', 'M2.wav', 'F1.wav', 'M4.wav', 'M5.wav', 'F5.wav', 'M1.wav']


In [2]:
def autocorr_method(frame, sfreq, threshold=0.56, fmin=50, fmax=350):
    """Estimate pitch using autocorrelation
    """

    
    # Calculate autocorrelation using scipy correlate
    frame = frame.astype(np.float)
    frame -= frame.mean()
    amax = np.abs(frame).max()
    if amax > 0:
        frame /= amax
    else:
        return 0

    corr = correlate(frame, frame)
    # keep the positive part
    corr = corr[len(corr)//2:]

    # Find the first minimum
    dcorr = np.diff(corr)
    rmin = np.where(dcorr > 0)[0]
    if len(rmin) > 0:
        rmin1 = rmin[0]
    else:
        return 0

    # Find the next peak
    peak = np.argmax(corr[rmin1:]) + rmin1
    rmax = corr[peak]/corr[0]
    f0 = sfreq / peak

    if rmax > threshold and f0 >= fmin and f0 <= fmax:
        return f0
    else:
        return 0

In [3]:
class Counters:
    def __init__(self, gross_threshold=0.2):
        self.num_voiced = 0
        self.num_unvoiced = 0
        self.num_voiced_unvoiced = 0
        self.num_unvoiced_voiced = 0
        self.num_voiced_voiced = 0
        self.num_gross_errors = 0
        self.fine_error = 0
        self.e2 = 0
        self.gross_threshold = gross_threshold
        self.nfiles = 0

    def add(self, other):
        if other is not None:
            self.num_voiced += other.num_voiced
            self.num_unvoiced += other.num_unvoiced
            self.num_voiced_unvoiced += other.num_voiced_unvoiced
            self.num_unvoiced_voiced += other.num_unvoiced_voiced
            self.num_voiced_voiced += other.num_voiced_voiced
            self.num_gross_errors += other.num_gross_errors
            self.fine_error += other.fine_error
            self.e2 += other.e2
            self.nfiles += 1

    def __repr__(self):
        nframes = self.num_voiced + self.num_unvoiced
        if self.nfiles > 0:
            self.fine_error /= self.nfiles
        str = [
            f"Num. frames:\t{self.num_unvoiced + self.num_voiced} = {self.num_unvoiced} unvoiced + {self.num_voiced} voiced",
            f"Unvoiced frames as voiced:\t{self.num_unvoiced_voiced}/{self.num_unvoiced} ({100*self.num_unvoiced_voiced/self.num_unvoiced:.2f}%)",
            f"Voiced frames as unvoiced:\t{self.num_voiced_unvoiced}/{self.num_voiced} ({100*self.num_voiced_unvoiced/self.num_voiced:.2f}%)",
            f"Gross voiced errors (>{100*self.gross_threshold}%):\t{self.num_gross_errors}/{self.num_voiced_voiced} ({100*self.num_gross_errors/self.num_voiced_voiced:.2f}%)",
            f"MSE of fine errors:\t{100*self.fine_error:.2f}%",
            f"RMSE:\t{np.sqrt(self.e2/nframes):.2f}"
        ]
        return '\n'.join(str)

In [4]:
def compare(fref, pitch):
    vref = np.loadtxt(fref)
    vtest = np.array(pitch)

    diff_frames = len(vref) - len(vtest)
    if abs(diff_frames) > 5:
        print(f"Error: number of frames in ref ({len(vref)}) != number of frames in test ({len(vtest)})")
        return None
    elif diff_frames > 0:
        vref = np.resize(vref, vtest.shape)
    elif diff_frames < 0:
        vtest = np.resize(vtest, vref.shape)

    counters = Counters()
    counters.num_voiced = np.count_nonzero(vref)
    counters.num_unvoiced = len(vref) - counters.num_voiced
    counters.num_unvoiced_voiced = np.count_nonzero(np.logical_and(vref == 0, vtest != 0))
    counters.num_voiced_unvoiced = np.count_nonzero(np.logical_and(vref != 0, vtest == 0))

    voiced_voiced = np.logical_and(vref != 0, vtest != 0)
    counters.num_voiced_voiced = np.count_nonzero(voiced_voiced)

    f = np.absolute(vref[voiced_voiced] - vtest[voiced_voiced])/vref[voiced_voiced]
    gross_errors = f > counters.gross_threshold
    counters.num_gross_errors = np.count_nonzero(gross_errors)
    fine_errors = np.logical_not(gross_errors)
    counters.fine_error = np.sqrt(np.square(f[fine_errors]).mean())
    counters.e2 = np.square(vref - vtest).sum()

    return counters

In [5]:
from scipy.signal import butter, lfilter, freqz, medfilt, hilbert
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def wav2f0(options, gui):
    fs = open(options.submission, 'w') if options.submission is not None else None
    totalCounters = Counters()
    with open(gui) as f:
        if fs is not None:
            print('id,frequency', file=fs)
        for line in f:
            line = line.strip()
            if len(line) == 0:
                continue
            filename = os.path.join(options.datadir, line + ".wav")
            f0ref_filename = os.path.join(options.datadir, line + ".f0ref")
            print("Processing:", filename)
            sfreq, data = wavfile.read(filename)
            nsamples = len(data)
            #plt.figure(figsize=(20,10))
            #plt.specgram(data, Fs=sfreq, NFFT=1024, noverlap=192, cmap='nipy_spectral', xextent=(0,len(data)))
            #plt.ylabel('Frequency [Hz]')
            #plt.xlabel('Time [sec]')
            #plt.show()

            # From miliseconds to samples
            ns_windowlength = int(round((options.windowlength * sfreq) / 1000))
            ns_frameshift = int(round((options.frameshift * sfreq) / 1000))
            ns_left_padding = int(round((options.left_padding * sfreq) / 1000))
            ns_right_padding = int(round((options.right_padding * sfreq) / 1000))
            pitch = []
            
            #Pre-processing
            cutoff_freq = 300
            n_coeficients = 10
            
            
            print(len(data))
            
            # Get the filter coefficients so we can check its frequency response.
            b, a = butter_lowpass(cutoff_freq, sfreq, n_coeficients)
            '''
            # Plot the frequency response.
            w, h = freqz(b, a, worN=8000)
            plt.subplot(2, 1, 1)
            plt.plot(0.5*sfreq*w/np.pi, np.abs(h), 'b')
            plt.plot(cutoff_freq, 0.5*np.sqrt(2), 'ko')
            plt.axvline(cutoff_freq, color='k')
            plt.xlim(0, 0.5*sfreq)
            plt.title("Lowpass Filter Frequency Response")
            plt.xlabel('Frequency [Hz]')
            plt.grid()
            plt.show()
            break
            '''
            # Filter the signal
            data = butter_lowpass_filter(data, cutoff_freq, sfreq, n_coeficients)
            
            
            cutoff_freq = 50
            n_coeficients = 7
            '''
            b, a = butter_highpass(cutoff_freq, sfreq, n_coeficients)
            w, h = freqz(b, a, worN=8000)
            plt.subplot(2, 1, 1)
            plt.plot(0.5*sfreq*w/np.pi, np.abs(h), 'b')
            plt.plot(cutoff_freq, 0.5*np.sqrt(2), 'ko')
            plt.axvline(cutoff_freq, color='k')
            plt.xlim(0, 200)
            plt.title("Highpass Filter Frequency Response")
            plt.xlabel('Frequency [Hz]')
            plt.grid()
            plt.show()
            break
            '''
            
            
            data = butter_highpass_filter(data, cutoff_freq, sfreq, n_coeficients)
            
            # Preprocessing (centre-clipping)
            centreclip = 200
            data = (abs(data) > centreclip).astype(np.int) * data
            
            #print(len(data))
            for id, ini in enumerate(range(-ns_left_padding, nsamples - ns_windowlength + ns_right_padding + 1, ns_frameshift)):
                first_sample = max(0, ini)
                last_sample = min(nsamples, ini + ns_windowlength)
                frame = data[first_sample:last_sample]
                f0 = autocorr_method(frame, sfreq)
                if fs is not None:
                    print(line + '_' + str(id) + ',', f0, file=fs)
                pitch.append(f0)
            # Post-processing
            medianwindow = 3
            pitch = medfilt(pitch,medianwindow)
            if os.path.isfile(f0ref_filename):
                counters = compare(f0ref_filename, pitch)
                totalCounters.add(counters)

    if totalCounters.num_voiced + totalCounters.num_unvoiced > 0:
        print("### Summary")
        print(totalCounters)
        print("-------------------------------\n")

In [6]:
fda_ue_options = SimpleNamespace(
    windowlength=32, frameshift=15, left_padding=16, right_padding=16, datadir='../input', submission=None)
wav2f0(fda_ue_options, '../input/fda_ue.gui')

Processing: ../input/fda_ue/rl001.wav
30000
Processing: ../input/fda_ue/rl002.wav
40000
Processing: ../input/fda_ue/rl003.wav
40000
Processing: ../input/fda_ue/rl004.wav
32000
Processing: ../input/fda_ue/rl005.wav
30000
Processing: ../input/fda_ue/rl006.wav
40000
Processing: ../input/fda_ue/rl007.wav
30000
Processing: ../input/fda_ue/rl008.wav
40000
Processing: ../input/fda_ue/rl009.wav


/opt/conda/lib/python3.6/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


32000
Processing: ../input/fda_ue/rl010.wav
50000
Processing: ../input/fda_ue/rl011.wav
40000
Processing: ../input/fda_ue/rl012.wav
34000
Processing: ../input/fda_ue/rl013.wav
34000
Processing: ../input/fda_ue/rl014.wav
30000
Processing: ../input/fda_ue/rl015.wav
60000
Processing: ../input/fda_ue/rl016.wav
42000
Processing: ../input/fda_ue/rl017.wav
30000
Processing: ../input/fda_ue/rl018.wav
24000
Processing: ../input/fda_ue/rl019.wav
30000
Processing: ../input/fda_ue/rl020.wav
24000
Processing: ../input/fda_ue/rl021.wav
60000
Processing: ../input/fda_ue/rl022.wav
60000
Processing: ../input/fda_ue/rl023.wav
60000
Processing: ../input/fda_ue/rl024.wav
60000
Processing: ../input/fda_ue/rl025.wav
60000
Processing: ../input/fda_ue/rl026.wav
60000
Processing: ../input/fda_ue/rl027.wav
60000
Processing: ../input/fda_ue/rl028.wav
100000
Processing: ../input/fda_ue/rl029.wav
60000
Processing: ../input/fda_ue/rl030.wav
80000
Processing: ../input/fda_ue/rl031.wav
60000
Processing: ../input/fda_

In [7]:
test_options = SimpleNamespace(
    windowlength=26.5, frameshift=10, left_padding=13.25, right_padding=7, datadir='../input/test', submission='submission.csv')
wav2f0(test_options, '../input/test.gui')

Processing: ../input/test/F1.wav
644201


/opt/conda/lib/python3.6/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


Processing: ../input/test/F2.wav
674001
Processing: ../input/test/F3.wav
610001
Processing: ../input/test/F4.wav
632001
Processing: ../input/test/F5.wav
774001
Processing: ../input/test/M1.wav
747501
Processing: ../input/test/M2.wav
637501
Processing: ../input/test/M3.wav
543501
Processing: ../input/test/M4.wav
674001
Processing: ../input/test/M5.wav
806001
